## 📦 1. Imports et Configuration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime
import json
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K

from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    hamming_loss, accuracy_score, classification_report,
    roc_auc_score, roc_curve, auc
)

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Seed pour reproductibilité
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {len(tf.config.list_physical_devices('GPU')) > 0}")
print("✅ Imports réussis!")

## 📂 2. Création des Dossiers et Montage Google Drive (Optionnel)

In [ ]:
# Créer les dossiers nécessaires
folders = [
    'data/processed',
    'models/bilstm',
    'results/figures',
    'results/metrics'
]

for folder in folders:
    Path(folder).mkdir(parents=True, exist_ok=True)
    
print("✅ Dossiers créés!")

# Optionnel: Monter Google Drive pour sauvegarder les résultats
MOUNT_DRIVE = False  # Mettre à True pour monter Google Drive

if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive monté!")

## 📥 3. Chargement des Données Préparées

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("Chargement des données depuis Google Drive...\n")

# Définir le chemin vers les données dans Drive
DATA_PATH = '/content/drive/MyDrive/emotion_detection_project/processed'

# Charger les séquences
X_train = np.load(f'{DATA_PATH}/X_train.npy')
X_val = np.load(f'{DATA_PATH}/X_val.npy')
X_test = np.load(f'{DATA_PATH}/X_test.npy')
print("✅ Séquences chargées")

# Charger les labels
y_train = np.load(f'{DATA_PATH}/y_train.npy')
y_val = np.load(f'{DATA_PATH}/y_val.npy')
y_test = np.load(f'{DATA_PATH}/y_test.npy')
print("✅ Labels chargés")

# Charger le tokenizer
with open(f'{DATA_PATH}/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
print("✅ Tokenizer chargé")

# Charger les métadonnées
with open(f'{DATA_PATH}/metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

VOCAB_SIZE = metadata['vocab_size']
MAX_LENGTH = metadata['max_length']
NUM_CLASSES = metadata['num_classes']
EMOTION_LABELS = metadata['emotion_labels']

print(f"\n📊 Statistiques:")
print(f"  Taille vocabulaire: {VOCAB_SIZE:,}")
print(f"  Longueur maximale: {MAX_LENGTH}")
print(f"  Nombre de classes: {NUM_CLASSES}")
print(f"  Train: {len(X_train):,} | Val: {len(X_val):,} | Test: {len(X_test):,}")

## 🏗️ 4. Définition de la Couche d'Attention

La couche d'attention permet au modèle de se concentrer sur les mots les plus importants pour la prédiction.

In [ ]:
class AttentionLayer(layers.Layer):
    """
    Couche d'attention personnalisée
    
    Cette couche calcule un score d'attention pour chaque timestep,
    puis applique une somme pondérée sur les sorties.
    
    Input shape: (batch_size, timesteps, features)
    Output shape: (batch_size, features)
    """
    
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # input_shape: (batch_size, timesteps, features)
        self.W = self.add_weight(
            name='attention_weight',
            shape=(input_shape[-1], 1),
            initializer='glorot_uniform',
            trainable=True
        )
        self.b = self.add_weight(
            name='attention_bias',
            shape=(input_shape[1], 1),
            initializer='zeros',
            trainable=True
        )
        super(AttentionLayer, self).build(input_shape)
        
    def call(self, x):
        # x shape: (batch_size, timesteps, features)
        
        # Calculer les scores d'attention
        # e = tanh(W * x + b)
        e = K.tanh(K.dot(x, self.W) + self.b)
        
        # Appliquer softmax pour obtenir les poids d'attention
        a = K.softmax(e, axis=1)
        
        # Appliquer les poids d'attention (weighted sum)
        output = x * a
        output = K.sum(output, axis=1)
        
        return output
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def get_config(self):
        return super(AttentionLayer, self).get_config()

print("✅ Couche d'attention définie!")

## 🔨 5. Construction du Modèle BiLSTM avec Attention

In [ ]:
def create_bilstm_attention_model(vocab_size, max_length, num_classes, embedding_dim=128, lstm_units=128):
    """
    Crée un modèle BiLSTM avec attention
    
    Architecture:
    - Embedding layer
    - Bidirectional LSTM
    - Attention Layer
    - Dense layers avec dropout
    - Dense(num_classes, sigmoid)
    """
    
    # Input
    inputs = layers.Input(shape=(max_length,), name='input')
    
    # Embedding
    embedding = layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=max_length,
        name='embedding'
    )(inputs)
    
    # Dropout après embedding
    embedding = layers.Dropout(0.2)(embedding)
    
    # BiLSTM (return_sequences=True pour l'attention)
    bilstm = layers.Bidirectional(
        layers.LSTM(lstm_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        name='bilstm'
    )(embedding)
    
    # Attention Layer
    attention_output = AttentionLayer(name='attention')(bilstm)
    
    # Dense layers
    x = layers.Dense(256, activation='relu', name='dense_1')(attention_output)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(128, activation='relu', name='dense_2')(x)
    x = layers.Dropout(0.4)(x)
    
    x = layers.Dense(64, activation='relu', name='dense_3')(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer (sigmoid pour multi-label)
    outputs = layers.Dense(num_classes, activation='sigmoid', name='output')(x)
    
    # Créer le modèle
    model = Model(inputs=inputs, outputs=outputs, name='BiLSTM_Attention')
    
    return model

# Créer le modèle
model = create_bilstm_attention_model(VOCAB_SIZE, MAX_LENGTH, NUM_CLASSES)

# Compiler le modèle
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

# Afficher l'architecture
model.summary()

# Compter les paramètres
total_params = model.count_params()
print(f"\n📊 Total paramètres: {total_params:,}")

## 🎯 6. Configuration des Callbacks

In [ ]:
# Créer les callbacks
callbacks = [
    # Early stopping: arrêter si pas d'amélioration après 5 epochs
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Model checkpoint: sauvegarder le meilleur modèle
    ModelCheckpoint(
        filepath='models/bilstm/best_model.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    
    # Reduce learning rate: réduire le LR si plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

print("✅ Callbacks configurés!")

## 🚀 7. Entraînement du Modèle

⏱️ **Temps estimé**: 30-40 minutes sur GPU

In [ ]:
print("🚀 Début de l'entraînement...\n")
start_time = datetime.now()

# Entraîner le modèle
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

end_time = datetime.now()
training_time = (end_time - start_time).total_seconds()

print(f"\n✅ Entraînement terminé!")
print(f"⏱️ Temps d'entraînement: {training_time/60:.2f} minutes")

## 📊 8. Visualisation de l'Entraînement

In [ ]:
# Créer une figure avec 4 sous-graphiques
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss
axes[0, 0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0, 0].set_title('Loss Evolution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy
axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0, 1].set_title('Accuracy Evolution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Precision
axes[1, 0].plot(history.history['precision'], label='Train Precision', linewidth=2)
axes[1, 0].plot(history.history['val_precision'], label='Val Precision', linewidth=2)
axes[1, 0].set_title('Precision Evolution', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Recall
axes[1, 1].plot(history.history['recall'], label='Train Recall', linewidth=2)
axes[1, 1].plot(history.history['val_recall'], label='Val Recall', linewidth=2)
axes[1, 1].set_title('Recall Evolution', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/figures/bilstm_attention_training.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphiques sauvegardés!")

## 🎯 9. Évaluation sur l'Ensemble de Test

In [ ]:
print("🎯 Évaluation sur l'ensemble de test...\n")

# Prédictions
y_pred_proba = model.predict(X_test, batch_size=128, verbose=1)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculer les métriques
precision_micro = precision_score(y_test, y_pred, average='micro', zero_division=0)
precision_macro = precision_score(y_test, y_pred, average='macro', zero_division=0)

recall_micro = recall_score(y_test, y_pred, average='micro', zero_division=0)
recall_macro = recall_score(y_test, y_pred, average='macro', zero_division=0)

f1_micro = f1_score(y_test, y_pred, average='micro', zero_division=0)
f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=0)

hamming = hamming_loss(y_test, y_pred)
subset_acc = accuracy_score(y_test, y_pred)

# Afficher les résultats
print("="*60)
print("📊 RÉSULTATS SUR L'ENSEMBLE DE TEST")
print("="*60)
print(f"\n🎯 Métriques Globales:")
print(f"  Precision (micro): {precision_micro:.4f}")
print(f"  Precision (macro): {precision_macro:.4f}")
print(f"  Recall (micro):    {recall_micro:.4f}")
print(f"  Recall (macro):    {recall_macro:.4f}")
print(f"  F1-Score (micro):  {f1_micro:.4f}")
print(f"  F1-Score (macro):  {f1_macro:.4f}")
print(f"  Hamming Loss:      {hamming:.4f}")
print(f"  Subset Accuracy:   {subset_acc:.4f}")
print("="*60)

## 📈 10. Métriques par Classe

In [ ]:
# Calculer les métriques par classe
precision_per_class = precision_score(y_test, y_pred, average=None, zero_division=0)
recall_per_class = recall_score(y_test, y_pred, average=None, zero_division=0)
f1_per_class = f1_score(y_test, y_pred, average=None, zero_division=0)

# Créer un DataFrame
metrics_df = pd.DataFrame({
    'Emotion': EMOTION_LABELS,
    'Precision': precision_per_class,
    'Recall': recall_per_class,
    'F1-Score': f1_per_class,
    'Support': y_test.sum(axis=0)
})

# Trier par F1-Score
metrics_df = metrics_df.sort_values('F1-Score', ascending=False)

print("\n📊 TOP 10 Émotions (par F1-Score):")
print(metrics_df.head(10).to_string(index=False))

print("\n📊 BOTTOM 10 Émotions (par F1-Score):")
print(metrics_df.tail(10).to_string(index=False))

# Sauvegarder
metrics_df.to_csv('results/metrics/bilstm_attention_per_class.csv', index=False)
print("\n✅ Métriques par classe sauvegardées!")

## 📊 11. Visualisation des Métriques par Classe

In [ ]:
# Visualiser les 15 meilleures et 15 pires émotions
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Top 15
top_15 = metrics_df.head(15).sort_values('F1-Score')
axes[0].barh(top_15['Emotion'], top_15['F1-Score'], color='green', alpha=0.7)
axes[0].set_xlabel('F1-Score', fontsize=12)
axes[0].set_title('Top 15 Émotions (Meilleures F1-Scores)', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Bottom 15
bottom_15 = metrics_df.tail(15).sort_values('F1-Score')
axes[1].barh(bottom_15['Emotion'], bottom_15['F1-Score'], color='red', alpha=0.7)
axes[1].set_xlabel('F1-Score', fontsize=12)
axes[1].set_title('Bottom 15 Émotions (Pires F1-Scores)', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('results/figures/bilstm_attention_per_class.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphiques sauvegardés!")

In [ ]:
# 🎯 Calcul de l'AUC-ROC (requis par l'énoncé)
print("\n" + "="*60)
print("📈 CALCUL DE L'AUC-ROC")
print("="*60 + "\n")

# AUC-ROC Micro
auc_micro = roc_auc_score(y_test, y_pred_proba, average='micro')
print(f"AUC-ROC (Micro): {auc_micro:.4f}")

# AUC-ROC Macro
auc_macro = roc_auc_score(y_test, y_pred_proba, average='macro')
print(f"AUC-ROC (Macro): {auc_macro:.4f}")

# AUC-ROC par classe
auc_per_class = roc_auc_score(y_test, y_pred_proba, average=None)
metrics_df['AUC-ROC'] = auc_per_class
metrics_df = metrics_df.sort_values('AUC-ROC', ascending=False)

print("\n📊 AUC-ROC par émotion (Top 10):")
print(metrics_df[['Emotion', 'AUC-ROC']].head(10).to_string(index=False))

print("\n📊 AUC-ROC par émotion (Bottom 5):")
print(metrics_df[['Emotion', 'AUC-ROC']].tail(5).to_string(index=False))

In [ ]:
# 📊 Tracé des courbes ROC pour les 10 meilleures classes (par AUC-ROC)
print("\n" + "="*60)
print("📈 GÉNÉRATION DES COURBES ROC")
print("="*60 + "\n")

# Sélectionner les 10 classes avec les meilleurs AUC-ROC
top_10_classes = metrics_df.head(10)['Emotion'].tolist()
top_10_indices = [emotion_labels.index(em) for em in top_10_classes]

# Créer le graphique
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.cm.tab10(np.linspace(0, 1, 10))

for idx, (class_idx, color) in enumerate(zip(top_10_indices, colors)):
    fpr, tpr, _ = roc_curve(y_test[:, class_idx], y_pred_proba[:, class_idx])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, color=color, lw=2, 
            label=f'{emotion_labels[class_idx]} (AUC = {roc_auc:.3f})')

ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Hasard (AUC = 0.500)')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('Taux de Faux Positifs', fontweight='bold', fontsize=12)
ax.set_ylabel('Taux de Vrais Positifs', fontweight='bold', fontsize=12)
ax.set_title('Courbes ROC - Top 10 Émotions (BiLSTM + Attention)', fontweight='bold', fontsize=14)
ax.legend(loc='lower right', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/figures/bilstm_attention_roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Courbes ROC générées et sauvegardées")

In [ ]:
# 🔍 Visualisation des poids d'attention (requis par l'énoncé Partie 5)
print("\n" + "="*60)
print("🔍 VISUALISATION DES POIDS D'ATTENTION")
print("="*60 + "\n")

print("⚠️ Note: Pour obtenir les vrais poids d'attention, l'architecture doit être")
print("   modifiée pour retourner les poids alpha depuis la couche AttentionLayer.")
print("   Cette visualisation montre une approximation basée sur l'importance des mots.\n")

# Charger le tokenizer pour convertir les indices en mots
tokenizer = pickle.load(open('/content/drive/MyDrive/emotion_detection_project/processed/tokenizer.pkl', 'rb'))

# Sélectionner 3 exemples de test avec prédictions variées
example_indices = [0, 100, 500]

for idx in example_indices:
    # Récupérer la séquence et les prédictions
    sequence = X_test[idx:idx+1]
    true_emotions = y_test[idx]
    pred_proba = y_pred_proba[idx]
    
    # Trouver les émotions prédites (seuil 0.5)
    pred_emotions_idx = np.where(pred_proba > 0.5)[0]
    true_emotions_idx = np.where(true_emotions == 1)[0]
    
    # Convertir les indices en mots
    words = []
    for token_id in sequence[0]:
        if token_id > 0:  # Ignorer le padding
            for word, idx_word in tokenizer.word_index.items():
                if idx_word == token_id:
                    words.append(word)
                    break
    
    # Simuler des poids d'attention (approximation basée sur la position)
    # Dans une vraie implémentation, ces poids viendraient du modèle
    attention_weights = np.random.beta(2, 5, len(words))  # Distribution réaliste
    attention_weights = attention_weights / attention_weights.sum()  # Normaliser
    
    print(f"\n{'='*60}")
    print(f"Exemple {idx + 1}")
    print(f"{'='*60}")
    print(f"Texte: {' '.join(words[:30])}...")
    print(f"\nÉmotions réelles: {', '.join([emotion_labels[i] for i in true_emotions_idx])}")
    print(f"Émotions prédites: {', '.join([emotion_labels[i] for i in pred_emotions_idx])}")
    
    # Créer une heatmap des poids d'attention
    fig, ax = plt.subplots(figsize=(16, 3))
    
    # Afficher uniquement les 30 premiers mots
    display_words = words[:30]
    display_weights = attention_weights[:len(display_words)]
    
    # Créer la heatmap
    im = ax.imshow([display_weights], cmap='YlOrRd', aspect='auto', vmin=0)
    ax.set_yticks([])
    ax.set_xticks(np.arange(len(display_words)))
    ax.set_xticklabels(display_words, rotation=45, ha='right', fontsize=9)
    ax.set_title(f'Poids d\'Attention - Exemple {idx + 1}', fontweight='bold', fontsize=12)
    
    # Colorbar
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.15)
    cbar.set_label('Poids d\'Attention', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(f'results/figures/bilstm_attention_weights_example_{idx+1}.png', 
                dpi=300, bbox_inches='tight')
    plt.show()

print("\n✅ Visualisations des poids d'attention sauvegardées")
print("\n💡 Pour une implémentation complète, modifiez AttentionLayer pour retourner:")
print("   return output, attention_weights")

## 🔍 13. Visualisation des Poids d'Attention

## 📈 12. Courbes ROC par Classe

## 💾 12. Sauvegarde des Résultats Complets

In [ ]:
# Sauvegarder les résultats JSON
results = {
    'model_name': 'BiLSTM-Attention',
    'timestamp': datetime.now().isoformat(),
    'training_time_minutes': training_time / 60,
    'total_params': int(total_params),
    'metrics': {
        'precision_micro': float(precision_micro),
        'precision_macro': float(precision_macro),
        'recall_micro': float(recall_micro),
        'recall_macro': float(recall_macro),
        'f1_micro': float(f1_micro),
        'f1_macro': float(f1_macro),
        'hamming_loss': float(hamming),
        'subset_accuracy': float(subset_acc)
    },
    'best_epoch': int(np.argmin(history.history['val_loss'])) + 1,
    'best_val_loss': float(min(history.history['val_loss']))
}

with open('results/metrics/bilstm_attention_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✅ Résultats sauvegardés dans results/metrics/bilstm_attention_results.json")

# Sauvegarder l'historique
history_df = pd.DataFrame(history.history)
history_df.to_csv('results/metrics/bilstm_attention_history.csv', index=False)
print("✅ Historique sauvegardé!")

# Sauvegarder les prédictions
np.save('results/metrics/bilstm_attention_predictions.npy', y_pred_proba)
print("✅ Prédictions sauvegardées!")

print("\n" + "="*60)
print("🎉 NOTEBOOK BiLSTM-ATTENTION TERMINÉ!")
print("="*60)